In [1]:
import time
import cv2

In [2]:
# carregando a rede neural
net = cv2.dnn.readNet('yolov4-tiny.weights', 'yolov4-tiny.cfg')

# criando o modelo a partir da rede neural
model = cv2.dnn.DetectionModel(net)

# setando os parametros de entrada para o modelo utilizado
model.setInputParams(size=(416,416), scale=(1/255))

In [3]:
# carregando os nomes para uma lista
class_names = []
with open('coco_names.txt','r') as f:
    class_names = [cname.strip() for cname in f.readlines()]

In [4]:
# definindo as cores de cada nome
name_color = []
for i in class_names:
    color =(((sum(map(ord,i))*2)%255),((sum(map(ord,i))*3)%255),((sum(map(ord,i))*5)%255))
    name_color.append(color)

In [5]:
cap =  cv2.VideoCapture(0)

In [6]:
# Defina as novas dimensões desejadas
nova_largura = 1920
nova_altura = 1040

#codigo de detecçao
while cap.isOpened():

    _, frame = cap.read()
    
    # Redimensione o frame para as novas dimensões
    frame = cv2.resize(frame, (nova_largura, nova_altura))
    
    start = time.time()
    classes, scores, boxes = model.detect(frame,0.1,0.2)
    end = time.time()

    for (classid, score, box) in zip(classes, scores, boxes):
        color = name_color[classid]
        label = f"{class_names[classid]}: {score}"
        
        cv2.rectangle(frame, box, color, 2)
        cv2.putText(frame, label, (box[0], box[1] - 10), cv2.FONT_HERSHEY_COMPLEX, 0.5, color, 2)
    
    fps_label = f"FPS: {round((1/(end - start)),2)}"

    cv2.putText(frame, fps_label, (0,25), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,0), 5)
    cv2.putText(frame, fps_label, (0,25), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 3)

    cv2.imshow("detections", frame)

    if cv2.waitKey(1) == 27:
        break

cap.release()
cv2.destroyAllWindows()